In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file.
city_data_df = pd.read_csv ("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,0,Sept-Iles,50.2001,-66.3821,40.05,71,42,5.01,CA,scattered clouds,2022-05-04 03:11:12
1,1,Ushuaia,-54.8000,-68.3000,38.86,70,20,6.91,AR,few clouds,2022-05-04 03:11:13
2,2,Nouakchott,18.0858,-15.9785,64.33,87,2,10.40,MR,clear sky,2022-05-04 03:11:14
3,3,Thinadhoo,0.5333,72.9333,79.54,88,100,20.71,MV,overcast clouds,2022-05-04 03:11:14
4,4,Mataura,-46.1927,168.8643,63.48,58,16,1.95,NZ,few clouds,2022-05-04 03:11:15


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 79
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)
# preferred_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
3,3,Thinadhoo,0.5333,72.9333,79.54,88,100,20.71,MV,overcast clouds,2022-05-04 03:11:14
10,10,Port Blair,11.6667,92.7500,82.44,78,40,6.91,IN,scattered clouds,2022-05-04 03:10:53
17,17,Vaini,-21.2000,-175.2000,80.76,69,75,8.05,TO,broken clouds,2022-05-04 03:11:19
25,25,Bengkulu,-3.8004,102.2655,82.89,70,94,4.29,ID,overcast clouds,2022-05-04 03:11:22
26,26,Flinders,-34.5833,150.8552,79.18,56,7,7.00,AU,clear sky,2022-05-04 03:06:42
27,27,Takoradi,4.8845,-1.7554,79.72,85,83,4.03,GH,broken clouds,2022-05-04 03:11:22
29,29,Meulaboh,4.1363,96.1285,79.56,82,100,6.80,ID,overcast clouds,2022-05-04 03:06:29
33,33,Victoria,22.2855,114.1577,82.96,58,16,10.00,HK,few clouds,2022-05-04 03:08:55
35,35,Carnarvon,-24.8667,113.6333,84.27,26,5,13.80,AU,clear sky,2022-05-04 03:11:08
39,39,Ciudad Guayana,8.3533,-62.6528,79.52,81,96,5.19,VE,overcast clouds,2022-05-04 03:11:26


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                125
City                   125
Lat                    125
Lng                    125
Max Temp               125
Humidity               125
Cloudiness             125
Wind Speed             125
Country                125
Current Description    125
Date                   125
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Thinadhoo,MV,79.54,overcast clouds,0.5333,72.9333,
10,Port Blair,IN,82.44,scattered clouds,11.6667,92.7500,
17,Vaini,TO,80.76,broken clouds,-21.2000,-175.2000,
25,Bengkulu,ID,82.89,overcast clouds,-3.8004,102.2655,
26,Flinders,AU,79.18,clear sky,-34.5833,150.8552,
27,Takoradi,GH,79.72,broken clouds,4.8845,-1.7554,
29,Meulaboh,ID,79.56,overcast clouds,4.1363,96.1285,
33,Victoria,HK,82.96,few clouds,22.2855,114.1577,
35,Carnarvon,AU,84.27,clear sky,-24.8667,113.6333,
39,Ciudad Guayana,VE,79.52,overcast clouds,8.3533,-62.6528,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("hotel isn't found skip the city.")
        

hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head(10)
#preferred_cities_df.count()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Thinadhoo,MV,79.54,overcast clouds,0.5333,72.9333,
10,Port Blair,IN,82.44,scattered clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
17,Vaini,TO,80.76,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
25,Bengkulu,ID,82.89,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
26,Flinders,AU,79.18,clear sky,-34.5833,150.8552,Shellharbour Resort & Conference Centre
27,Takoradi,GH,79.72,broken clouds,4.8845,-1.7554,Raybow International Hotel
29,Meulaboh,ID,79.56,overcast clouds,4.1363,96.1285,Kost Pria
33,Victoria,HK,82.96,few clouds,22.2855,114.1577,Mini Hotel Central
35,Carnarvon,AU,84.27,clear sky,-24.8667,113.6333,Hospitality Carnarvon
39,Ciudad Guayana,VE,79.52,overcast clouds,8.3533,-62.6528,Hotel Las Americas


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")



In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:


# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))